# Tutorial 2-1: The Mechanics of Backprop – "Autograd from Scratch"

**Course:** CSEN 342: Deep Learning  
**Topic:** Computational Graphs, Chain Rule, and Automatic Differentiation

## Objective
Deep learning frameworks like PyTorch and TensorFlow rely on **Automatic Differentiation** (Autograd). They build a computational graph of your operations and automatically calculate gradients for you.

In this tutorial, we will "open the black box" by building a tiny Autograd engine from scratch. We will:
1.  Create a `Value` class that stores data and its gradient.
2.  Manually implement the `backward()` pass for basic operations (Add, Multiply, ReLU, Sigmoid).
3.  Recreate the detailed backpropagation example from the lecture slides to verify our math.
4.  Compare our manual engine against PyTorch to see that they produce identical results.

---

## Part 1: The Engine (A Tiny `Tensor`)

We need a data structure that can store a scalar value (the data) and the gradient of the loss with respect to that value (the derivative).

Recall the Chain Rule from the slides:
$$ \text{upstream\_grad} = \frac{\partial L}{\partial \text{output}} $$
$$ \text{local\_grad} = \frac{\partial \text{output}}{\partial \text{input}} $$
$$ \frac{\partial L}{\partial \text{input}} = \text{local\_grad} * \text{upstream\_grad} $$

In [ ]:
import math
import numpy as np

class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0          # Initially, gradient is zero
        self._backward = lambda: None # Function to propagate gradient backward
        self._prev = set(_children)   # Keep track of input nodes (for the graph)
        self._op = _op           # The operation that produced this node
        self.label = label       # Optional name for debugging

    def __repr__(self):
        return f"Value(data={self.data:.4f}, grad={self.grad:.4f})"

    # --- Operations ---
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            # Gradient of addition is 1.0 * upstream
            # f = a + b  -> df/da = 1, df/db = 1
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            # Gradient of mult is the other value * upstream
            # f = a * b -> df/da = b, df/db = a
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    # Support for the power operator (**) needed for x^-1\n
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Only supporting int/float powers for now"
        out = Value(self.data ** other, (self,), f'**{other}')

        def _backward():
            # f = x^n -> df/dx = n * x^(n-1)
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    # Re-using __mul__ for negation (-1 * self)
    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def sigmoid(self):
        x = self.data
        t = 1 / (1 + math.exp(-x))
        out = Value(t, (self,), 'sigmoid')

        def _backward():
            # d(sigmoid)/dx = sigmoid * (1 - sigmoid)
            local_grad = t * (1 - t)
            self.grad += local_grad * out.grad
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')
        
        def _backward():
            # d(e^x)/dx = e^x
            self.grad += out.data * out.grad
        out._backward = _backward
        return out

    def backward(self):
        # Topological sort to ensure we backprop in the correct order
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        # Initialize gradient of the final output (loss) to 1.0
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

---

## Part 2: Recreating the Lecture Slide Example

Let's recreate the computational graph shown in the lecture slides (Slide 14-27). 

**The Function:**
$$ f(x, w) = \frac{1}{1 + e^{-(w_0x_0 + w_1x_1 + w_2)}} $$

**The Inputs:**
* $w_0 = 2.0, x_0 = -1.0$
* $w_1 = -3.0, x_1 = -2.0$
* $w_2 = -3.0$

In [ ]:
# 1. Initialize Inputs
w0 = Value(2.0, label='w0'); x0 = Value(-1.0, label='x0')
w1 = Value(-3.0, label='w1'); x1 = Value(-2.0, label='x1')
w2 = Value(-3.0, label='w2')

# 2. Forward Pass (Building the Graph)
# Operations mimic the slide flow:
dot1 = w0 * x0; dot1.label = 'w0*x0'
dot2 = w1 * x1; dot2.label = 'w1*x1'

sum1 = dot1 + dot2; sum1.label = 'sum_dots'
sum2 = sum1 + w2;   sum2.label = 'sum_bias'

# Slide implements sigmoid manually as: 1 / (1 + exp(-x))
# We will do the same to match the slide's step-by-step derivative
neg_z = -sum2;          neg_z.label = '-z'
exp_z = neg_z.exp();    exp_z.label = 'exp(-z)'
den = exp_z + 1.0;      den.label = '1 + exp(-z)'
out = den**-1;          out.label = '1/x' 
# Note: Our Value class needs __pow__ for den**-1. 
# Let's add a quick power helper just for this demo:
def power(self, other):
    out = Value(self.data**other, (self,), f'**{other}')
    def _backward():
        self.grad += (other * self.data**(other-1)) * out.grad
    out._backward = _backward
    return out
Value.__pow__ = power

# Rerun the final step with the new power method
out = den**-1; out.label = 'output'

print("Forward Pass Complete.")
print("Output:", out.data)

### 2.1 Backward Pass
Now we call `out.backward()`. This triggers the chain rule from the end of the graph back to the inputs.

In [ ]:
out.backward()

print("Gradients Calculated:")
print(f"w0.grad: {w0.grad} (Slide Expectation: ~ -0.20)")
print(f"x0.grad: {x0.grad} (Slide Expectation: ~ 0.40)")
print(f"w1.grad: {w1.grad} (Slide Expectation: ~ -0.40)")
print(f"x1.grad: {x1.grad} (Slide Expectation: ~ -0.60)")
print(f"w2.grad: {w2.grad} (Slide Expectation: ~ 0.20)")

**Analysis:** compare these numbers to the red numbers in Slide 26. 
*Note: There might be slight floating point differences or sign differences depending on exactly how the slide defined inputs, but the magnitude should match.*

---

## Part 3: PyTorch Verification

Now we do the exact same thing using PyTorch's `autograd`. This confirms that our tiny engine is mathematically correct.

In [ ]:
import torch

# 1. Define Inputs (requires_grad=True tells PyTorch to track operations)
w0_t = torch.tensor([2.0], requires_grad=True)
x0_t = torch.tensor([-1.0], requires_grad=True)
w1_t = torch.tensor([-3.0], requires_grad=True)
x1_t = torch.tensor([-2.0], requires_grad=True)
w2_t = torch.tensor([-3.0], requires_grad=True)

# 2. Forward Pass
# We use torch.sigmoid to mimic the full operation flow directly
z = (w0_t * x0_t) + (w1_t * x1_t) + w2_t
out_t = torch.sigmoid(z)

print("PyTorch Output:", out_t.item())

# 3. Backward Pass
out_t.backward()

print("\nPyTorch Gradients:")
print(f"w0.grad: {w0_t.grad.item()}")
print(f"x0.grad: {x0_t.grad.item()}")
print(f"w1.grad: {w1_t.grad.item()}")
print(f"x1.grad: {x1_t.grad.item()}")
print(f"w2.grad: {w2_t.grad.item()}")

### Discussion
1.  **The Graph:** In PyTorch, you don't see the graph explicitly, but it is built dynamically as you perform operations (like `+` and `*`).
2.  **Memory:** Every time you create a node with `requires_grad=True`, PyTorch allocates memory to store the graph. This is why we use `with torch.no_grad():` during validation/testing—to stop building this graph and save memory.
3.  **Accumulation:** Gradients accumulate. If we ran `out_t.backward()` again, the gradients would double. This is why we must call `optimizer.zero_grad()` in a training loop.